In [1]:
import os
import sys
import librosa
import numpy as np
from tqdm import tqdm

sys.path.insert(0, "/home/splinter/Desktop/VoiceMed/CoughCNN/short-spectro/")

import detector

cough = detector.Detector()


/home/splinter/.local/lib/python3.8/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/home/splinter/.local/lib/python3.8/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


In [2]:
DATASET = "dataset/"
WEBAPP = "/home/splinter/Desktop/VoiceMed/qualityCheck/data/web_app/"
OUT = "data/positive/"

In [4]:
print("Processing WebApp data")
for folder in os.listdir(WEBAPP):
    if os.path.isdir(WEBAPP+folder):
        for sample in os.listdir(WEBAPP+folder):
            #since labels are wrong no point filtering for cough in names
            if os.path.splitext(sample)[-1] != ".wav" or "covid" not in sample:
                continue

            cough.detect(WEBAPP+folder+"/"+sample, OUT, False)

Processing WebApp data


In [3]:
print("Processing VocoAI data")
folder = DATASET+"voco/"
for sample in tqdm(os.listdir(folder)):
    if os.path.splitext(sample)[-1] != ".wav":
        continue

    cough.detect(folder+sample, OUT, False)

  0%|          | 0/33 [00:00<?, ?it/s]

Processing VocoAI data


100%|██████████| 33/33 [00:10<00:00,  3.16it/s]


In [5]:
print("Processing Hospital data")
folder = "/home/splinter/Desktop/VoiceMed/Hospital/CleanedHospitalSamples/"
for patient in tqdm(os.listdir(folder)):
    for sample in os.listdir(folder+patient):
        if os.path.splitext(sample)[-1] != ".wav" and "cough" in sample:
            continue

        cough.detect(folder+patient+"/"+sample, OUT, False)

  0%|          | 0/8 [00:00<?, ?it/s]

Processing Hospital data


100%|██████████| 8/8 [00:26<00:00,  3.31s/it]


Did also some manual cleaning as a couple false positives where there due to the sligth inaccuracy of the cough detector

In [6]:
folder = "data/negative/"
for sample in tqdm(os.listdir(folder)):
    if os.path.splitext(sample)[-1] != ".wav":
        continue
    
    cough.detect(folder+sample, folder, False)    
    os.system("rm '"+folder+sample+"'")

100%|██████████| 28/28 [02:59<00:00,  6.40s/it]


In [8]:
SR = 44000
N_FFT = 2048
#N_MELS = 60
N_MELS = 64

def melspectrogram(signal):
    signal = librosa.util.normalize(signal)
    spectro = librosa.feature.melspectrogram(
        signal,
        sr=SR,
        n_mels=N_MELS,
        n_fft=N_FFT
    )
    spectro = librosa.power_to_db(spectro)
    spectro = spectro.astype(np.float32)
    return spectro    

Investigating the "t" dimension of the melspectrogram and how to get it to a precise value. Turns out that the ratio between "t" and the length in seconds of the signal is constant and is equal to 86. This investigation was carried in order to fix the decoder problem which scaled the spectrogram to higher dimensions due to the rounding of the inital dimension when running the convolution.

In [24]:
signal, sr = librosa.load("data/positive/316.wav", sr=SR)
print(len(signal)/SR)
print(melspectrogram(signal).shape)

0.5
(64, 43)


In [25]:
signal, sr = librosa.load("/home/splinter/Desktop/VoiceMed/qualityCheck/bad_sample.wav", sr=SR)
print(len(signal)/SR)
print(melspectrogram(signal).shape)

11.572931818181818
(64, 995)


The dimensions needed for the autoencoder to work are [64, 48, 1]. Which can be acheived with a sample leng of:

In [27]:
print(48/86, "seconds")

0.5581395348837209 seconds


In [9]:
LABELS = ["positive", "negative"]

data = []
folder = "data/"

for i, label in enumerate(LABELS):    
    for sample in tqdm(os.listdir(folder+label+"/")):
        if os.path.splitext(sample)[-1] != ".wav":
            continue

        signal, sr = librosa.load(folder+label+"/"+sample, sr=SR)
        data.append([i, melspectrogram(signal)])

np.random.shuffle(data)
np.save("dataset.npy", data)

100%|██████████| 3596/3596 [00:07<00:00, 493.75it/s]
